In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Dropout, ActivityRegularization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.regularizers import l1

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

from tensorflow.keras import regularizers

from tensorflow.keras.applications import VGG16



In [2]:

# Download the train, validation and test dataset which were created with the dataset preprocess files

# file_id_audio = [
#     '1NNxYN2DbuDF0tvzbZTQ_EPEJ5uIPZPai', # https://drive.google.com/file/d/1NNxYN2DbuDF0tvzbZTQ_EPEJ5uIPZPai/view?usp=sharing
#     '1-CMJZY4allQBba5gl5s0Y-RT9Qdma_Mj', # https://drive.google.com/file/d/1-CMJZY4allQBba5gl5s0Y-RT9Qdma_Mj/view?usp=sharing
#     '1--LAbXygf83JJuY-Do3tG_6cVxSErHr5', # https://drive.google.com/file/d/1--LAbXygf83JJuY-Do3tG_6cVxSErHr5/view?usp=sharing
# ]


# for i, id in enumerate(file_id_audio):
#   # Check if the file already exists
#   if (not os.path.exists('val_dataset.tfrecord')) or (not os.path.exists('train_dataset.tfrecord')) or (not os.path.exists('test_dataset.tfrecord')):
#     !wget --content-disposition 'https://drive.google.com/uc?export=download&id={id}&confirm=t'




In [3]:
# # Another way to get the dataset from the Google Drive and the working enviroment is Google Colab 

# from google.colab import drive
# drive.mount('/content/drive')

# import shutil

# drive_file_path = '/content/drive/MyDrive/test_dataset.tfrecord'
# colab_file_path = '/content/test_dataset.tfrecord'

# shutil.copy(drive_file_path, colab_file_path)

# drive_file_path = '/content/drive/MyDrive/val_dataset.tfrecord'
# colab_file_path = '/content/val_dataset.tfrecord'

# shutil.copy(drive_file_path, colab_file_path)


# drive_file_path = '/content/drive/MyDrive/train_dataset.tfrecord'
# colab_file_path = '/content/train_dataset.tfrecord'

# shutil.copy(drive_file_path, colab_file_path)

In [4]:

# Function to parse TFRecord entries, declaring the metadatas of the TFRecord files
def _parse_function(proto):
    keys_to_features = {
        'data': tf.io.FixedLenFeature([128 * 313 * 3], tf.float32),
        'labels': tf.io.FixedLenFeature([], tf.int64),
    }
    parsed_features = tf.io.parse_single_example(proto, keys_to_features)
    parsed_features['data'] = tf.reshape(parsed_features['data'], (128, 313, 3))
    return parsed_features['data'], parsed_features['labels']





In [5]:
# Load and parse TFRecord datasets for testing, validation,and training
# These dataset are pipelined so they are not loaded to the memory.

test_dataset = tf.data.TFRecordDataset('test_dataset2.tfrecord').map(_parse_function)
val_dataset = tf.data.TFRecordDataset('val_dataset2.tfrecord').map(_parse_function)
train_dataset = tf.data.TFRecordDataset('train_dataset2.tfrecord').map(_parse_function)

In [6]:
# Create a ResNet model with a pre-trained ResNet50 base

resnet_model = Sequential()
pretrained_model= tf.keras.applications.VGG19(
    include_top=False,
    input_shape=(128,313,3),
    pooling='avg',classes=264,
    weights='imagenet')

# Freeze pre-trained ResNet50 layers
for layer in pretrained_model.layers:
        layer.trainable=False


resnet_model.add(pretrained_model)


resnet_model.add(Flatten())

resnet_model.add(BatchNormalization())



# # Add Dense layers with dropout and L2 regularization
# resnet_model.add(Dense(1024, activation='relu',kernel_regularizer=keras.regularizers.l1(0.00001)))
# resnet_model.add(BatchNormalization())
# resnet_model.add(Dropout(0.2))


# # Add Dense layers with dropout and L2 regularization
# resnet_model.add(Dense(1024, activation='relu',kernel_regularizer=keras.regularizers.l1(0.00001)))
# resnet_model.add(BatchNormalization())
# resnet_model.add(Dropout(0.2))


# resnet_model.add(Dense(512, activation='relu',kernel_regularizer=keras.regularizers.l1(0.00001)))
# resnet_model.add(BatchNormalization())
# resnet_model.add(Dropout(0.2))

# resnet_model.add(Dense(1024, activation='relu',kernel_regularizer=keras.regularizers.l2(0.001) ))
# resnet_model.add(BatchNormalization())
# resnet_model.add(Dropout(0.3))






resnet_model.add(Dense(1024, activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001)))
resnet_model.add(BatchNormalization())
resnet_model.add(Dropout(0.5))


resnet_model.add(Dense(1024, activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001)))
resnet_model.add(BatchNormalization())
resnet_model.add(Dropout(0.5))

resnet_model.add(Dense(1024, activation='relu',kernel_regularizer=keras.regularizers.l2(0.0001)))
resnet_model.add(BatchNormalization())
resnet_model.add(Dropout(0.5))



resnet_model.add(Dense(264, activation='softmax'))

In [7]:
# Compile the model

optimizer = Adam(learning_rate=0.001)

resnet_model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Display model summary
pretrained_model.summary()


Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 313, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 313, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 313, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 156, 64)       0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 156, 128)      73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 156, 128)      147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 78, 128)       0     

In [8]:
# Prefetch datasets for better performance
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

# Shuffle training dataset
train_dataset = train_dataset.shuffle(5)



In [9]:

# Define early stopping and checkpoint callbacks
patience = 20
early_stopping=EarlyStopping(patience=patience, verbose=2)
checkpointer=ModelCheckpoint(filepath='weights.hdf5', save_best_only=True, verbose=2)

In [ ]:
# Train the model
num_epochs = 30
batch_size = 128

history = resnet_model.fit(
    train_dataset.batch(batch_size),
    epochs=num_epochs,
    validation_data=val_dataset.batch(batch_size),
    callbacks=[checkpointer, early_stopping]
)

Epoch 1/30
395/395 [==============================] - 346s 814ms/step - loss: 5.9420 - accuracy: 0.0503 - val_loss: 4.9893 - val_accuracy: 0.0837

Epoch 00001: val_loss improved from inf to 4.98931, saving model to weights.hdf5
Epoch 2/30
395/395 [==============================] - 295s 746ms/step - loss: 5.1075 - accuracy: 0.1078 - val_loss: 4.5669 - val_accuracy: 0.1530

Epoch 00002: val_loss improved from 4.98931 to 4.56685, saving model to weights.hdf5
Epoch 3/30
395/395 [==============================] - 299s 757ms/step - loss: 4.7104 - accuracy: 0.1500 - val_loss: 4.4038 - val_accuracy: 0.1791

Epoch 00003: val_loss improved from 4.56685 to 4.40381, saving model to weights.hdf5
Epoch 4/30
395/395 [==============================] - 312s 788ms/step - loss: 4.4751 - accuracy: 0.1836 - val_loss: 4.3139 - val_accuracy: 0.1918

Epoch 00004: val_loss improved from 4.40381 to 4.31394, saving model to weights.hdf5
Epoch 5/30
395/395 [==============================] - 299s 756ms/step - loss

In [ ]:
# # Train the model
# num_epochs = 30
# batch_size = 256

# history = resnet_model.fit(
#     train_dataset.batch(batch_size),
#     epochs=num_epochs,
#     validation_data=val_dataset.batch(batch_size),
#     #callbacks=[checkpointer, early_stopping]
# )

In [ ]:
# Plot training and validation loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.grid()
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
# # Get the accuracy on the test dataset

# evaluation_result = resnet_model.evaluate(test_dataset.batch(batch_size))

# loss_value = evaluation_result[0]

# accuracy_value = evaluation_result[1]


In [ ]:
# print(f'the test accuracy is {accuracy_value}')